## TODO:
1. Use other tools (t-sne, co-sne, poincare maps) to compute embeddings of tree data, mnist, c_elegans (maybe other data?) and see what they produce

2. Assuming they fail to produce results where hierarchy is visible, use that as motivation for my work.

3. Think of how to motivate their failure using the concept that the gradient is wrong.

4. Update MARE dates

# Current thing I'm working on

1. BH Approx. acceleration doesn't work yet for gaussianKL
2. We get NaN's from the cython file I think. What experiment reproduces this problem?


### HyperbolicKL notes
I'm trying to figure out what exactly is going wrong when certain flags are set:
1. Baseline experiment: (MNIST, wrong_grad, no scale_fix, HyperbolicKL, accel. on/off) \
   Works perfectly good. We get the results matching the paper (baseline)

2. Experiment 2: (MNIST, wrong_grad, scale_fix, HyperbolicKL, acce. on/off) \
   Works fairly well. We get results similar to baseline. But it requires (much) more iterations to converge nicely.
   This is due to the scaling of the inverse metric tensor term. Since it basically scales the gradients down consistently.

3. Experiment 3: (MNIST, correct_gad, no scale_fix, HyperbolicKL, accel. on/off) \
   Takes much longer to converge properly. The extra d^H_ij term makes gradients (near) 0 for a long time.
   Gradients get very large too at some point (embeddings overshoot the Disk)

4. Experiment 4: (MNIST, correct_grad, scale_fix, HyperbolicKL, accel. on/off) \
   Takes even longer to converge properly. Now we have both d^H_ij (initially) and scale_fix scaling the gradients down. \
   However now gradients stay within reasonable amounts and don't explode off to large values. Presumably due to the scale_fix \
   These settings reproduce the result where points are pushed towards the boundary (conceptual sensible one)

In points (3, 4) we get other issues. So the fact that gradients are very small means no progress is made for a long time, means the algorithm just stops.
\
If we use both fixes (gradient, scale_fix) then we get the "issue" that points are pushed towards the boundary. 

To avoid early stopping increase **n_iter_check** flag in **opt_config** and manually change **n_iter_without_progress** in **solver.py**


### GaussianKL Notes
GaussianKL works great on tree-data (hyperbolic var=0.1, "simple" tree, no accel.). However, applied to regular data we get some issues:
\
All experiments below use a hyperbolic var of 0.1. MNIST(np=0.025, perp=25, pca=50)

1. Gradients are extremely small when no early exaggeration is turned on.

2. Without early exaggeration, we need a lot of iterations (25k - experiment 34, 35) to convergence. It does converge nicely though.

3. With early exaggeration, we get much nicer embeddings? **TBD**

4. **[NOTE!!]** In the original GaussianKL cost function (python function), there was an additional **grad \*= 4** statement which was a renmnant of the HyperbolicKL code. This lead to much speedier convergence. Perhaps for practical reasons, we would want to include some gradient scaling (or learning rate adjustment tbh??) when it comes to this kind of stuff.
\
\
It seems that the problem has been resolved. I suspect the problem was in an undeclared variable in the cython code (mult or dij_sq) which is now fixed. We don't get weird values anymore

5. I think there are still issues with the GaussianKL:
\
**1**: In the gradient ($p_{ij} - q^H_{ij}$) term can act badly based on $p_{ij}$. Because I suspect we are dealing with space issues in the end. Therefore it might be very important to derive some (heuristical) relationship between Hyperbolic Gaussian $\sigma^2$ and Euclidean embedding width and Euclidean $\sigma^2$
\
**2**: Numerical issues occur for some reason. Sometimes (often), gradient shoots up to extremely large values. I'm not sure yet why this occurs.
\
**3**: Specifically, it seems that small depth. large nr. of children works fine. But larger depth poses issues (Space issues?)
\
**4 [FIXED]**: GaussianKL BH approx. produces numerical issues. Cython code was missing declaration of local thread buffers in the| negative BH term of the GaussianKL
\
**5**: Tree data that with high depth results in strange embeddings.
\
**6 [FIXED]**: BH. embeddings need more iterations before early stopping.
\
**7 [FIXED]**: GaussianKL embeddings are highly dependent on the variance value. Can we find some heuristic for this?
\
**8**: For GaussianKL, scaling the gradient uniformly by some constant seems to help some embeddings converge nicer and faster. However, this is not effective in general. Different kinds of data sets, size, tree depth, etc.. may require different scalings? Therefore we can't really rely on this

We build the tree **qt.build_tree(pos_output)** when we **not exact == True**
- Is **QuadTree** correcly constructed? 
- Could there be a problem with initial embeddings that are all close to eachother, causing numerical issues
  when using BH approx. ?
- How is the **Polar QuadTree** build? How does it exactly work?
  Need to understand building process
- How is it used for speeding up computations? 
  Need to understand summarizing process
- Is there a way we can "hack" things so they're more suitable for our use case?

Then we compute **negative term** using **QuadTree**
- How is **QuadTree** used to summarize force terms?
- Why **neg_force[ax] += mult * summary[j * offset + ax]** in BH approx. negative term? Shouldn't this be a multiplication with the gradient of the distance (i.e. gradient of summary), and not the distance (summary)? -> **A:** In the quadtree, the summary holds the hyperbolic gradient distance so everything works out fine

2. Code investigation/study

I would like to understand the code a bit better. The design choices etc.. for the polar quadtree construction, gradient calculation.

- distance_grad returns a vector value. However, the implementation returns a scalar, and it seems like we're basically computing things as if things were a vector equation, but only returning the scalar of the axis that we desire. This probably leads to redundant computations though. (Although, maybe not since using vectors fundamentally requires extra computations?)

# Gaussian vs t-Distribution 

### Introduction
The goal of hyperbolic visualizations is to reveal hierarchical structure in the data. 
This means we have some room to distort the embeddings as long as hierarchical structure is kept as that is our primary goal.

The hope is that embedding in Hyperbolic SPace naturally reveals hierarchical structure. But when we do this naively using hyperbolic t-sne, points are pushed along the boundary of the Poincare Disk, distorting the visaulization. 

Most likely hierarchical structure is stil present conceptually/theoretically, but it is hard to derive this from the visualization itself as all points would be along the boundary.
Hence we hope that by restricting the width of the embedding to a portion of the Poincare Disk where we cn still visually discern things, hierarchical structure can still emerge.
- From experiments using tree-data it seems to be empirically valid?

### Why does hpyerbolic t-sne not work well?
To get visually pleasant results, we want that points on opposites sides (near opposite boundary of the Poincare Disk) have small or no neighbour probability. Meaning that, conceptually these points represnt very distant points. However, in practice, the (hyperbolic) distance between 2 points on opposite sides is quite small. (Which is conceptually/theoretically correct since most of Hyperbolic space is along hte boundary and we can only visualize a tiny portion of the space). 

But for visualizations to be useful, we want to restrict the full visualization within that tiny space. The t-distribution distance function (cost function) is not fit for this as it makes probabilities fall off too slow. Points at opposite sides (ex. x=-0.99, x=0.99) still have high probability since they're technically not that distant. So this means that repulsive forces are still very strong when we don't want them to be strong.

Hence we want to use a distance function that falls off much quicker. This is where the use of a Gaussian function comes in. Since the Gaussian function has exponentially decaying tails, and a variance parameter that we can control, we can (manually) restrict our embeddings to some desired size.

### Is the use of a Gaussian justified?
Originally the t-distribution replaced a gaussian function due to the "Crowding Problem". However, in Hyperbolic space we can argue that, since space expands exponentially (although we do restrict ourselves to a tiny fraction of the space, this argument is not very solid), we can go back to using the Gaussian function as we won't run into the Crowding problem. 

### Hyperbolic embeddings vs. Visualizing Hyperbolic embeddings

# Hyperbolic space gradient analysis

In the T-SNE paper, the t-distribution is introduced to fight against the crowding problem. So if in high dimensions we have n equidistant points, then in 2 (embedding) dimensions, we can't faithfully capture this equidistance and instead points will get squished together.

\
\
**Mathematically:**

$p_{ij} > 0$ get represented by $q_{ij} = 0$ because we can't faithfully model $y_i$ and $y_j$ to be close due to a lower dimensional embedding.

So then when we update along the direction $(y_i - y_j)$ with (spring) factor $(p_{ij} - q_{ij})$, because $q_{ij} = 0$, we only have a attractive force due to $(p_{ij} - q_{ij}) = (p_{ij} - 0) > 0$.

To make the points not crush eachother into a small area (crowding), we use a t-distribution for the lower dimensional embeddings to ensure that we can still faithfully capture the same high dimensional probabilities (using less low dimensional space), by spreading the points in low dimensions out more since we are using a heavy-tailed distribution to match affinities.

\
\
\
**Hyperbolic case:**

In the hyperbolic scenario we might not need to use the t-distribution since space is expanded exponentially from the origin. This means that we do not have the crowding problem since a point in low dimensional space has enough space to accommodate for many equidistant points in high dimensional space due to the exponentially expanding nature of hyperbolic space.

Potential experiment: Use regular gaussian instead of t-distribution

# Correct gradient contribution to behaviour

**What is the contribution factor of the extra $d^{H}_{ij}$ in the gradient?**

Enlargers the gradient of the cost function. If 2 points are far apart this factor is big. So it further accentuates repulsion or attraction depending on $(p_{ij} - q_{ij})$. Causes points to attract or repel more.

Hypothesis: 
1. $d^{H}_{ij}$ probably speeds up convergence by increasing the attractive/repulsive forces initially when it matters, (when $(p_{ij} - q_{ij})$ is not approximately 0 yet)
2. Might introduce more numerical instability, practically we might be ofshooting the edge of the unit circle. (Requires more investigation)


\
\
**Why is it here in the hyperbolic scenario but not in the euclidean case?**

Probably as an artifact from using hyperbolic space, so we need an extra factor? Investigate analytically


# Interpreting Hyperbolic embeddings

**Why do we even want hyperbolic embeddings?**
- We think hyperbolic geometry is the underlying geometrical structure for hierarchical data
- Naturally models tree-like (hierarchical) data due to the nature of its exponentially expanding space


**How should we interpret visualizations on the Poincare Disk?**
- We expect clusters inside the disk, with clusters expanding outwards from the center. 
- The expansion outwards indicates hierarchical links between clusters closer to the center, and the clusters further out
- Clusters near the edge of the disk are very far apart from clusters in the center -> Hierarchical ordering
- Clusters near the border are actually far apart from other eachother even if they seem close on the disk


# Notes on Poincare Embeddings for Gene Expressions

Biology, and especially in the field of genetics, gene expression, it is common to want to understand the relationships between things. This is done through visualization, data analysis, etc..
Here a brief overview of how it works, how people generally proceed, and the why's.

**What is gene expression?**
- Gene expression refers to a part of the process of how genetic material turns into observable traits
  The most fundamental level at which the genotype (genetic material) gives rise to the phenotype (observable traits)

**What is this process of gene expression?**
- DNA:                contains your genetic material
- Gene activation:    a step where specific genes in your DNA are activated 
- Transcription:      here the activated gene(s) get copied and stored into a molecule called mRNA
- Translation:        the step where the mRNA molecule is used to build a protein
- Protein:            a protein is a complex molecule that has specific uses/tasks to fulfill. This directly affects the phenotype

**Why visualize gene expression in hyperbolic space?**
- Naturally contains hierarchical structure. We can basically assume this?

# Thesis notes

### [1] Some notes regarding what I've run and observations

**Planaria Data - 90% of data - Correct gradient - Approximate:**\
No instances of numerical instabilities have occurred yet as far as I know

**Planaria Data - 40% of data - Correct gradient - Exact:**\
No numerical instability issues afaik


### [2] Structure of the code
Running the algorithm involves the following stages:
1. Initialize **HyperbolicTSNE** object and run the **.fit_transform()** method

2. **.fit_transform()** calls the (private) **._fit()** method 

3. **HyperbolicTSNE** takes in an Optimizer object. In the default case this will be the SequentialOptimizer class

4. **SequentialOptimizer** requires a “config” to run properly. These are set at the beginning (before .1), through a dedicated function for initializing the config. 

- In the default case **.sequence_poincare()** The SequentialOptimizer works by chunking the optimization process into sequences. We can specify each chunk of the sequence through objects (a dictionary) which contains all the info for each block to run properly. These objects/blocks follow a standard format that must be adhered to We get two blocks in the SequentialOptimizer: EarlyExaggeration block and a GradientDescent block through the functions **add_block_early_exaggeration()** and **add_block_gradient_descent_with_rescale_and_gradient_mask()**

- Each block must internally contain information (see the add_block functions) specifying the optimization strategy to use (e..g gradient descent), and parameters. The optimization method is specified under the “function” dictionary key,  with the “params” key containing the params for the optimization method.

5. SequentialOptimizer has a **.run()** method which executes the function (optimization method). By default **.run()** calls the **.gradient_descent()** function in **solver.py**

6. The optimization method is specified in the **solver.py** file. 
Most code here deals with administrative/logging tasks, and other higher level functionality regarding optimization. The default implementation is roughly split into the following parts:
- Checks and initializing logging- Gradient computation (using cython implementation of lower level details)
- Logging of optimization step- Convergence checks The actual computation of the optimization step is split in several parts:
    Higher level detail in optimization method, cost function details in **cost_functions.py**, lower level details in tsne.pyx

7. **cost_functions.py** contains the wrapper code that calls the lower level implementation for computing the gradient. We can find the code here for executing exact, or bh-tsne gradient descent. 

8. **tsne.pyx** contains the low level details of the gradient computation. Here we also find the code for the quad-tree datastructure, exponential map, and other calculation functionality.


### [2.1] Adding new cost functions (with different parameters)
How to add a new cost function and make it compatible with the pre-existing structure.

How to use the config/params dicts to get it to do what we want: 
- Constructing the **HyperbolicTSNE()** object requires **opt_method** and **opt_params** arguments.
- We can specify a custom **opt_method** and its associated **opt_params** 
- By default this is **SequentialOptimizer** which has **HyperbolicKL** hardcoded into it.
  So we should modify this or come up with a new optimizer class
- I need to explicitly update the params for the cost function to include the new embedding y_i
- Another thing; The data is worked on flatenned. (**.ravel()**) is used during optimization. So make sure to account for that

Currently the way to feed extra parameters into the CostFunction classes (to its constructor when it gets constructed) is to append a dictionary containg (k,v) pars of parameter and its value to the opt_params field in Experiment. This is a very ugly way of doing it but the current best solution ---> Refactor at some point?


### [3] Turning on/off acceleration structure
In the **opt_config**, we can set a boolean for the **exact** flag. We can specify exact or an approximate computation of the gradient.

# Results analysis

**Mnist dataset**
Results for incorrect and correct gradient look the same